In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:1"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0.3):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(3, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1, corner_points = None, initial_triangulation = None):
        """
        corner_points, etc., shoudn't include a batch dimension
        """
        if corner_points == None:
            ncornerpoints = 4
        else:
            ncornerpoints = corner_points.size(0)
        if npoints <= ncornerpoints:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - ncornerpoints, 3], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - ncornerpoints, 3)
        )
        if corner_points == None:
            self.corner_points = torch.tensor(
                [[0, 0, 0], [3, 0, 0], [0, 3, 0], [0, 0, 3]], dtype = floattype, device=device
            )
        else:
            self.corner_points = corner_points
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 3]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, ncornerpoints], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - ncornerpoints], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """
        points are now triples
        triangles are now quadruples
        edges are now still just indices, but there are four of them per 'triangle', and they correspond to triples of points, not pairs
        we use  0,2,1  0,3,2  0,1,3  1,2,3  as the order of the four 'edges'/faces
        opposite face is always ordered such that the last two indices are swapped
        faces are always read ANTICLOCKWISE
        
        first three points of tetrahedron MUST be read clockwise (from the outside) to get correct sign on incircle test
        
        new point will be inserted in zeroth position, so if corresponding face of REMOVED tetrahedron is [x,y,z] (being read anticlockwise from outside in) new tetrahedron is [p, x, y, z]
        """
        
        """
        number of tetrahedra is not the same for each batch (in 3D), so store as a big list, and remember batch index that it comes from
        """
        if corner_points == None:
            initial_triangulation = torch.tensor([[0, 1, 2, 3]], dtype=torch.long, device=device)
        
        self.partial_delaunay_triangles = initial_triangulation.unsqueeze(0).expand(self.batchsize, -1, -1).reshape(-1, 4)
        self.batch_index = torch.arange(self.batchsize, dtype = torch.long, device = device).unsqueeze(1).expand(-1, initial_triangulation.size(0)).reshape(-1)
        
        self.batch_triangles = self.partial_delaunay_triangles.size(0) #[0]
        self.ntriangles = torch.full([self.batchsize], initial_triangulation.size(0), dtype = torch.long, device = device) #[self.batchsize]
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert point_index.size(0) == self.batchsize
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        triangles_coordinates = self.points[self.batch_index.unsqueeze(1), self.partial_delaunay_triangles] # [batch_triangles, 4, 3]
        
        newpoint = self.points[self.batch_index, point_index[self.batch_index]] # [batch_triangles, 3]

        incircle_matrix = torch.cat(
            [
                newpoint.unsqueeze(1),
                triangles_coordinates,
            ],
            dim=-2,
        )  # [batch_triangles, 5, 3]
        incircle_matrix = torch.cat(
            [
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                incircle_matrix,
                torch.ones([self.batch_triangles, 5, 1], dtype = floattype, device=device),
            ],
            dim=-1,
        )  # [batch_triangles, 5, 5]
        assert incircle_matrix.dtype == floattype
        assert str(incircle_matrix.device) == device
        
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batch_triangles], is True if inside incircle
        
        conflicts = incircle_test.sum()
        
        conflicting_triangles = self.partial_delaunay_triangles[incircle_test] # [conflicts, 4]
        
        conflicting_edges_index0 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([0, 0, 0, 1])
        conflicting_edges_index0 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index1 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([2, 3, 1, 2])
        conflicting_edges_index1 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index2 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([1, 2, 3, 3])
        conflicting_edges_index2 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges = torch.cat([conflicting_edges_index0.view(-1).unsqueeze(-1), conflicting_edges_index1.view(-1).unsqueeze(-1), conflicting_edges_index2.view(-1).unsqueeze(-1)], dim = -1).reshape(-1, 3) # [conflicts * 4, 3]
        
        edge_batch_index = self.batch_index[incircle_test].unsqueeze(1).expand(-1, 4).reshape(-1) # [conflicts * 4]
        
        indices = torch.LongTensor([0, 2, 1])
        comparison_edges = conflicting_edges[:, indices] # [conflicts * 4, 3]        
        
        unravel_nomatch_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        i = 1
        while True:
            
            todo_mask = unravel_nomatch_mask[:-i].logical_and(edge_batch_index[:-i] == edge_batch_index[i:])
            if i % 4 == 0:
                if todo_mask.sum() == 0:
                    break
            
            match_mask = todo_mask.clone()
            match_mask[todo_mask] = (conflicting_edges[:-i][todo_mask] != comparison_edges[i:][todo_mask]).sum(-1).logical_not()
            
            unravel_nomatch_mask[:-i][match_mask] = False
            unravel_nomatch_mask[i:][match_mask] = False
            
            i += 1
        
        batch_newtriangles = unravel_nomatch_mask.sum()
        
        nomatch_edges = conflicting_edges[unravel_nomatch_mask] # [batch_newtriangles, 3], already in correct order to insert into 1,2,3 (since already anticlockwise from outside in)
        assert list(nomatch_edges.size()) == [batch_newtriangles, 3]
        nomatch_batch_index = edge_batch_index[unravel_nomatch_mask] # [batch_newtriangles]
        
        nomatch_newpoint = point_index[nomatch_batch_index] # [batch_newtriangles]
        
        newtriangles = torch.cat([nomatch_newpoint.unsqueeze(1), nomatch_edges], dim = -1) # [batch_newtriangles, 4]
        
        
        nremoved_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        nnew_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        
        indices = self.batch_index[incircle_test]
        nremoved_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        indices = edge_batch_index[unravel_nomatch_mask]
        nnew_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        assert (nnew_triangles <= 2 * nremoved_triangles + 2).logical_not().sum().logical_not()
        
        """
        NOTE:
        I THINK it's possible for nnew_triangles to be less than nremoved_triangles (or my code is just buggy...)
        """
        
        assert nnew_triangles.sum() == batch_newtriangles
        assert nremoved_triangles.sum() == incircle_test.sum()
        
        nadditional_triangles = nnew_triangles - nremoved_triangles # [batchsize]
        ntriangles = self.ntriangles + nadditional_triangles # [batchsize]
        
        partial_delaunay_triangles = torch.empty([ntriangles.sum(), 4], dtype = torch.long, device = device)
        batch_index = torch.empty([ntriangles.sum()], dtype = torch.long, device = device)
        
        cumulative_triangles = torch.cat([torch.zeros([1], dtype = torch.long, device = device), nnew_triangles.cumsum(0)[:-1]]) # [batchsize], cumulative sum starts at zero
        
        """
        since may actually have LESS triangles than previous round, we insert all that survive into the first slots (in that batch)
        """
        good_triangle_indices = torch.arange(incircle_test.logical_not().sum(), dtype = torch.long, device = device)
        good_triangle_indices += cumulative_triangles[self.batch_index[incircle_test.logical_not()]]
        bad_triangle_indices_mask = torch.ones([ntriangles.sum(0)], dtype = torch.bool, device = device)
        bad_triangle_indices_mask.scatter_(0, good_triangle_indices, False)
        
        assert good_triangle_indices.size(0) == incircle_test.logical_not().sum()
        assert bad_triangle_indices_mask.sum() == batch_newtriangles
        
        partial_delaunay_triangles[good_triangle_indices] = self.partial_delaunay_triangles[~incircle_test]
        batch_index[good_triangle_indices] = self.batch_index[~incircle_test]
        
        partial_delaunay_triangles[bad_triangle_indices_mask] = newtriangles
        batch_index[bad_triangle_indices_mask] = nomatch_batch_index
        
        self.partial_delaunay_triangles = partial_delaunay_triangles
        self.batch_index = batch_index
        
        self.ntriangles = ntriangles
        self.batch_triangles = self.partial_delaunay_triangles.size(0)
        
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )
        
        self.cost += nremoved_triangles
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 30000, npoints = 14, batchsize = 100, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(4, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 1
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        loss = ( 
                - logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
                * positive_reinforcement_binary 
                + logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 2) 
                * negative_reinforcement_binary 
        ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
        

In [ ]:
train(npoints = 9, batchsize = 4000, nsamples = 8) #greedyprob - 2, with +ve reinforcement

15.129250526428223 -4.358691692352295 -4.765614986419678 -4.739549160003662 -4.73954963684082
14.350750923156738 -4.216161251068115 -4.741217136383057 -4.248434066772461 -4.248434066772461
14.019001007080078 -3.7978932857513428 -4.602450370788574 -3.9776740074157715 -3.9776737689971924
13.813750267028809 -3.3773114681243896 -4.405120372772217 -3.4763989448547363 -3.476398468017578
13.842500686645508 -3.4978816509246826 -4.472866535186768 -4.681652545928955 -4.681652545928955
13.887500762939453 -3.518005132675171 -4.4477152824401855 -3.8643157482147217 -3.8643157482147217
13.751750946044922 -3.4375264644622803 -4.400801658630371 -3.4367055892944336 -3.4367055892944336
13.691750526428223 -3.397784948348999 -4.40134859085083 -4.468914031982422 -4.468914031982422
13.735000610351562 -3.471248149871826 -4.445864677429199 -3.5519356727600098 -3.5519356727600098
13.706501007080078 -3.476353168487549 -4.444706916809082 -3.9627742767333984 -3.9627745151519775
13.600000381469727 -3.37321710586547

12.755250930786133 -3.2478106021881104 -4.511528968811035 -3.8587212562561035 -3.8587212562561035
12.821250915527344 -3.2739670276641846 -4.52323055267334 -3.115457773208618 -3.1154580116271973
12.748750686645508 -3.1606011390686035 -4.464600563049316 -4.83443546295166 -4.834434986114502
12.706501007080078 -3.069707155227661 -4.406120777130127 -4.9572434425354 -4.9572434425354
12.700750350952148 -3.0734715461730957 -4.428430080413818 -2.514915943145752 -2.514915943145752
12.685750961303711 -3.140763759613037 -4.469371795654297 -3.828078508377075 -3.828078508377075
12.690751075744629 -3.1678545475006104 -4.4834184646606445 -3.3737311363220215 -3.3737311363220215
12.702750205993652 -3.2027130126953125 -4.497316360473633 -3.2730226516723633 -3.2730226516723633
12.634500503540039 -3.171773672103882 -4.480669975280762 -5.099351406097412 -5.099350929260254
12.620750427246094 -3.116985559463501 -4.4475016593933105 -3.6983909606933594 -3.698390483856201
12.70150089263916 -3.115279197692871 -4.

12.46150016784668 -3.0711045265197754 -4.428197860717773 -5.221912860870361 -5.221912860870361
12.463750839233398 -3.095719337463379 -4.438640594482422 -4.726437568664551 -4.726437568664551
12.373750686645508 -3.0722157955169678 -4.445199966430664 -4.7885518074035645 -4.788551330566406
12.428500175476074 -3.0780093669891357 -4.433333873748779 -5.189404010772705 -5.189404010772705
12.455000877380371 -3.1071207523345947 -4.454606533050537 -3.142012596130371 -3.142012596130371
12.455750465393066 -3.111246347427368 -4.463281154632568 -5.190155982971191 -5.190155506134033
12.394750595092773 -3.121063709259033 -4.468566417694092 -3.6896533966064453 -3.6896533966064453
12.377750396728516 -3.1109206676483154 -4.459593772888184 -4.466289043426514 -4.466289043426514
12.428000450134277 -3.0883774757385254 -4.432188510894775 -5.035434246063232 -5.035434246063232
12.348000526428223 -3.0578620433807373 -4.432519912719727 -5.345159530639648 -5.345160007476807
12.433000564575195 -3.089921474456787 -4.

12.326251029968262 -3.043316125869751 -4.421884536743164 -2.7743732929229736 -2.7743735313415527
12.351000785827637 -3.0823428630828857 -4.452088356018066 -5.31668758392334 -5.316687107086182
12.293000221252441 -3.090081214904785 -4.440445899963379 -4.8821868896484375 -4.882186412811279
12.336750984191895 -3.121166229248047 -4.459181308746338 -2.5329113006591797 -2.5329113006591797
12.359500885009766 -3.06013560295105 -4.446451187133789 -5.4990620613098145 -5.4990620613098145
12.310750961303711 -3.034160852432251 -4.413632392883301 -4.707761764526367 -4.707761764526367
12.300750732421875 -3.048596143722534 -4.437098026275635 -3.0340652465820312 -3.0340652465820312
12.281750679016113 -3.071037530899048 -4.440008163452148 -5.7858686447143555 -5.7858686447143555
12.31100082397461 -3.108409881591797 -4.4672322273254395 -4.630233287811279 -4.6302337646484375
12.317750930786133 -3.099287748336792 -4.459866523742676 -3.188549757003784 -3.188549518585205
12.335250854492188 -3.0904111862182617 

12.307000160217285 -3.0596110820770264 -4.430300235748291 -3.3631064891815186 -3.3631062507629395
12.280000686645508 -3.067105531692505 -4.4405083656311035 -4.735889434814453 -4.735889434814453
12.307000160217285 -3.069979190826416 -4.44004487991333 -4.9980010986328125 -4.998001575469971
12.250000953674316 -3.0691590309143066 -4.436642646789551 -5.41336727142334 -5.41336727142334
12.257000923156738 -3.050647258758545 -4.429159641265869 -3.0951266288757324 -3.0951268672943115
12.219500541687012 -3.0220401287078857 -4.421276092529297 -5.228062152862549 -5.228061676025391
12.253750801086426 -3.0265069007873535 -4.426665782928467 -5.294299125671387 -5.294299602508545
12.296500205993652 -3.0164883136749268 -4.412742614746094 -3.8979713916778564 -3.8979711532592773
12.211250305175781 -3.054856777191162 -4.450938701629639 -4.862722873687744 -4.862722873687744
12.19325065612793 -3.0768136978149414 -4.457082748413086 -5.292364597320557 -5.292364597320557
12.224250793457031 -3.11930251121521 -4.

...

11.82175064086914 -2.99894642829895 -4.446615219116211 -4.283143997192383 -4.283143997192383
11.845500946044922 -2.975893259048462 -4.4319915771484375 -4.615304946899414 -4.615304946899414
11.810500144958496 -2.959980010986328 -4.424742221832275 -3.1125941276550293 -3.1125943660736084
11.780000686645508 -2.9311442375183105 -4.421549320220947 -3.577090263366699 -3.577090263366699
11.77025032043457 -2.949575901031494 -4.422210693359375 -4.951164245605469 -4.951164722442627
11.804250717163086 -2.95448637008667 -4.435777187347412 -3.2516422271728516 -3.2516422271728516
11.798250198364258 -2.9713199138641357 -4.450873374938965 -4.928890705108643 -4.928891181945801
11.786750793457031 -2.981217384338379 -4.4510979652404785 -3.8133203983306885 -3.8133203983306885
11.807500839233398 -2.9991226196289062 -4.463107585906982 -3.7408604621887207 -3.7408604621887207
11.768250465393066 -2.996192455291748 -4.464652061462402 -4.805944442749023 -4.805944442749023
11.837000846862793 -3.027055263519287 -4.

11.791000366210938 -2.9401967525482178 -4.425978660583496 -4.844257354736328 -4.844257831573486
11.801750183105469 -2.9638638496398926 -4.4361186027526855 -4.589942932128906 -4.589942932128906
11.82650089263916 -2.968712568283081 -4.437421798706055 -4.128998279571533 -4.128997802734375
11.82650089263916 -2.9707159996032715 -4.4412617683410645 -4.505125045776367 -4.505125522613525
11.8412504196167 -2.9722354412078857 -4.443474769592285 -4.543554306030273 -4.543554306030273
11.823750495910645 -2.9585556983947754 -4.4464497566223145 -6.2249040603637695 -6.224903583526611
11.785000801086426 -2.928492307662964 -4.418937683105469 -3.8241677284240723 -3.8241677284240723
11.832000732421875 -2.9299323558807373 -4.417084217071533 -5.826119422912598 -5.826119422912598
11.797750473022461 -2.930884599685669 -4.419473171234131 -4.739625453948975 -4.739625930786133
11.813250541687012 -2.952725887298584 -4.431689262390137 -4.151985168457031 -4.151985168457031
11.818500518798828 -2.968109130859375 -4.4

11.83025074005127 -2.966046094894409 -4.4390740394592285 -5.407749176025391 -5.407748699188232
11.803250312805176 -2.923693895339966 -4.41775369644165 -2.938253879547119 -2.93825364112854
11.76400089263916 -2.9041907787323 -4.4099202156066895 -4.521973609924316 -4.521973133087158
11.82925033569336 -2.9251739978790283 -4.41657829284668 -5.127601623535156 -5.127601146697998
11.83750057220459 -2.94647216796875 -4.425572872161865 -3.5057458877563477 -3.5057458877563477
11.795001029968262 -2.945316791534424 -4.425448417663574 -5.329431533813477 -5.329431533813477
11.757250785827637 -2.9431822299957275 -4.424561500549316 -5.328668594360352 -5.328668594360352
11.857501029968262 -2.939856767654419 -4.430355072021484 -5.119697570800781 -5.119697093963623
11.764250755310059 -2.923178195953369 -4.428382396697998 -4.822620391845703 -4.822619915008545
11.735250473022461 -2.9238829612731934 -4.42678689956665 -3.2210497856140137 -3.2210495471954346
11.774250984191895 -2.9367294311523438 -4.4385719299

11.821500778198242 -2.9949283599853516 -4.452287197113037 -4.769191741943359 -4.769191741943359
11.837750434875488 -3.0123109817504883 -4.4474897384643555 -5.2322998046875 -5.232300281524658
11.801750183105469 -2.972230911254883 -4.430778503417969 -4.577487945556641 -4.577487945556641
11.761750221252441 -2.9534718990325928 -4.424883842468262 -2.580415725708008 -2.580415964126587
11.796000480651855 -2.9493253231048584 -4.421748638153076 -4.880023002624512 -4.880023002624512
11.803000450134277 -2.9469287395477295 -4.424344062805176 -3.418416738510132 -3.4184162616729736
11.756750106811523 -2.9471089839935303 -4.433712482452393 -4.352344512939453 -4.352344512939453
11.787500381469727 -2.961581230163574 -4.425384998321533 -4.942139148712158 -4.942139625549316
11.826000213623047 -2.9740357398986816 -4.441967964172363 -5.235071659088135 -5.235071182250977
11.863750457763672 -2.975080728530884 -4.444296360015869 -3.3974523544311523 -3.397451877593994
11.811750411987305 -2.9574055671691895 -4.

11.781500816345215 -2.898498058319092 -4.405541896820068 -3.571192502975464 -3.571192741394043
11.82650089263916 -2.9050889015197754 -4.4145612716674805 -4.713814735412598 -4.713814735412598
11.794000625610352 -2.915536642074585 -4.414853096008301 -4.3869948387146 -4.386994361877441
11.781000137329102 -2.9477956295013428 -4.426809310913086 -5.012652397155762 -5.0126519203186035
11.779500961303711 -2.9697964191436768 -4.433642864227295 -3.0724644660949707 -3.0724644660949707
11.797000885009766 -3.0068228244781494 -4.465003490447998 -5.502713680267334 -5.502713680267334
11.858000755310059 -3.0346877574920654 -4.464147567749023 -5.171995162963867 -5.171995639801025
11.785250663757324 -2.9834508895874023 -4.443271636962891 -4.465429306030273 -4.465429782867432
11.741000175476074 -2.952226161956787 -4.433685302734375 -4.511773109436035 -4.511773109436035
11.791250228881836 -2.9168784618377686 -4.4099884033203125 -3.5497660636901855 -3.5497658252716064
11.801250457763672 -2.9224658012390137 

In [4]:
%%bash
nvidia-smi

Sun Mar 14 15:48:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.118.02   Driver Version: 440.118.02   CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:86:00.0 Off |                    0 |
| N/A   45C    P0   123W / 250W |  12959MiB / 16280MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   27C    P0    32W / 250W |  15325MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        